In [1]:
import mne
import pyedflib
import numpy as np
import matplotlib

In [2]:
%matplotlib osx

In [3]:
# load data file
filepath = 'data/chb01_04.edf'

f = pyedflib.EdfReader(filepath)
n = f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n, f.getNSamples()[0]))
for i in np.arange(n):
    sigbufs[i, :] = f.readSignal(i)

ch_names = f.getSignalLabels()
ch_freqs = f.getSampleFrequencies()
f.close()

In [4]:
print(ch_names)
print(ch_freqs)

['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8']
[256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256
 256 256 256 256 256]


**Q:** What should be done with duplicate channels?

In [6]:
data = np.zeros_like(sigbufs)
for i in range(sigbufs.shape[0]):
    data[i,:] = sigbufs[i,:]
data.shape

(23, 921600)

In [26]:
# convert from microvolt to volt
fun = lambda x: x*1e-6
data = fun(sigbufs)

In [32]:
info = mne.create_info(ch_names, sfreq=ch_freqs[0], ch_types=['eeg']*data.shape[0])
info['bads'] = ['T8-P8-0']
print(info)

<Info | 8 non-empty values
 bads: 1 items (T8-P8-0)
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: unspecified
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
>


<ipython-input-32-1aca00eaa8ab>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names, sfreq=ch_freqs[0], ch_types=['eeg']*data.shape[0])


In [33]:
raw = mne.io.RawArray(data, info)

Creating RawArray with float64 data, n_channels=23, n_times=921600
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.


In [37]:
plot_kwargs = {
    'scalings': dict(eeg=20e-5),   # zooms the plot out
}

In [38]:
# plot interictal
plot = raw.plot(start=0, duration=30.0, **plot_kwargs)

In [47]:
# plot seizure
# start: 1467s, end: 1494s
plot = raw.plot(start=1467.0, duration=27.0, **plot_kwargs)

Channels marked as bad: ['T8-P8-0']
